In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pprint import pprint

from dataclasses import dataclass
import ase
import io
from multiprocessing import Pool

In [ ]:
import plotly
plotly.offline.init_notebook_mode()

In [ ]:
structures = pd.read_feather('../data/structures.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

In [ ]:
methane_name = 'dsgdb9nsd_000001'
methane = structures[structures.molecule_name == methane_name]

ethanol_name = 'dsgdb9nsd_000014'
ethanol = structures[structures.molecule_name == ethanol_name]

In [ ]:
from chemistry import *

In [ ]:
from chemistry import *

def make_molecule(name, molecule_df):
    atoms = []
    for i, atom in molecule_df.iterrows():
        atoms.append(Atom(atom.atom, (atom.x, atom.y, atom.z)))
        
    return Molecule(name, atoms)

molecule_name = 'dsgdb9nsd_037488'
molecule = make_molecule(molecule_name, structures[structures.molecule_name == molecule_name])
print(compute_path(molecule, 12, 3))

plt_plot_molecule(molecule)
#ase_plot_molecule(molecule)
# plotly_plot_molecule(molecule)
ipv_plot_molecule(molecule)

pprint(molecule)


In [ ]:
from chemistry import compute_path
molecule = make_molecule('ethanol', ethanol)
compute_path(molecule, 0, 8)

In [ ]:
from chemistry import *

def read_molecule(molecule_name):
    global structures
    molecule_df = structures[structures.molecule_name == molecule_name]
    molecule = make_molecule(molecule_name, molecule_df)
    
#     molecule_df = labelled[labelled.molecule_name == molecule.name]
#     for _, row in molecule_df.iterrows():
#         path = compute_path(molecule, row.atom_index_0, row.atom_index_1)
#         if path is None or len(path) > 4:
#             raise Exception(f'{molecule.name} {row.atom_index_0} {row.atom_index_1} {row.type} {path} {row} failed')
            
#     molecule_df = unlabelled[unlabelled.molecule_name == molecule.name]
#     for _, row in molecule_df.iterrows():
#         path = compute_path(molecule, row.atom_index_0, row.atom_index_1)
#         if path is None or len(path) > 4:
#             raise Exception(f'{molecule.name} {row.atom_index_0} {row.atom_index_1} {row.type} {path} {row} failed')
            
    return molecule

def read_molecules(data):
    molecule_names = data.molecule_name.unique()
    with Pool(8) as pool:
        molecules = pool.imap_unordered(read_molecule, molecule_names, 1000)
        molecule_map = {molecule.name:molecule for molecule in molecules}
     
    return molecule_map
        
molecules = read_molecules(structures)

In [ ]:
len(molecules), len(structures.molecule_name.unique())

In [ ]:
import pickle

with open('../data/molecules.pickle', 'wb') as f:
    pickle.dump(molecules, f)